In [5]:
from catboost import CatBoostRegressor
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipynb.fs.full.utils as utils
import ipynb.fs.full.features as features
import ipynb.fs.full.features_distancias as f_distancias


In [6]:
df_train = pd.read_csv('./data/train_filtrado.csv')
df_eval = pd.read_csv('./data/test.csv')

df_train, df_eval = features.features_de_csvs(df_train, df_eval)
df_train, df_test = utils.dividir_df_testeo(df_train, test_size=0.01)

df_test = features.llenar_nulls(df_test, hgb_mean=True, df_fill=df_train)
df_train = features.llenar_nulls(df_train, hgb_mean=True)


In [7]:
df_test_f = features.features_independientes_precio(df_test)
df_test_f = features.features_dependientes_precio(df_test_f, df_train)

df_train_f = features.features_independientes_precio(df_train)
df_train_f = features.features_dependientes_precio(df_train_f, df_train)

df_test_f, cols_tipodepropiedad_ohe = features.columna_a_ohe(df_test_f, 'tipodepropiedad', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_provincia_ohe = features.columna_a_ohe(df_test_f, 'provincia', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_zona_ohe = features.columna_a_ohe(df_test_f, 'zona', df_aux=df_train_f, devolver_cols=True)

df_train_f = features.columna_a_ohe(df_train_f, 'tipodepropiedad', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'provincia', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'zona', df_aux=df_test_f)


df_train_f['fecha'] = pd.to_datetime(df_train_f['fecha']).astype(int)
df_test_f['fecha'] = pd.to_datetime(df_test_f['fecha']).astype(int)

# df_train_f = df_train_f.sample(frac=1).reset_index(drop=True)

df_train_idf = pd.read_csv('./data/train_idf.csv')
df_test_idf = pd.read_csv('./data/test_idf.csv')

df_train_f = pd.merge(df_train_f, df_train_idf, on= 'id', how= 'left')
df_test_f = pd.merge(df_test_f, df_test_idf, on= 'id', how= 'left')

df_train_f = f_distancias.feature_distancias(df_train_f)
df_test_f = f_distancias.feature_distancias(df_test_f, df_train_f)

In [60]:
df_train_f['habitaciones'] = df_train_f['habitaciones'].astype('int64')
df_train_f['banos'] = df_train_f['banos'].astype('int64')
df_train_f['metroscubiertos'] = df_train_f['metroscubiertos'].astype('int64')
df_train_f['metrostotales'] = df_train_f['metrostotales'].astype('int64')
df_train_f['count_id_zona'] = df_train_f['count_id_zona'].astype('int64')
df_train_f['distancia_centro_mexico'] = df_train_f['distancia_centro_mexico'].astype('int64')


x_train.in

0    3
1    6
2    1
3    0
4    1
Name: distancia_centro_mexico, dtype: int64

In [62]:
features = ['habitaciones', 'garages','banos','antiguedad', 'metroscubiertos',  'metrostotales',  'provincia', 'tipodepropiedad',
            'gimnasio', 'usosmultiples', 'piscina']


features_test = ['top_provincia','anio', 'prop_frecuente','zona', 'es_ciudad_centrica',
                 'count_id_zona', 'count_ciudad', 'puntaje', 
               'count_tipo_propiedad_ciudad','dias_desde_datos','meses_desde_datos',
                'intervalo_metros_totales', 'intervalo_metros_cubiertos', 'distancia_centro_mexico'
                ]

features += features_test

x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features, test_size=0.01)


# train_labels = 'precio'
# Initialize CatBoostRegressor
model = CatBoostRegressor(iterations=300,
                             learning_rate=0.02,
                             depth=10,
                             eval_metric='MAE',
                             random_seed = 23,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 1,
                             od_wait=100,
                             l2_leaf_reg= 4
                         )
# Fit model
model.fit(x_train, y_train,
        cat_features=features,
          eval_set=(x_test, y_test),
          plot=True)

CatBoostError: Invalid type for cat_feature[non-default value idx=0,feature_idx=2]=20.0 : cat_features must be integer or string, real number values and NaN values should be converted to string.

In [30]:
y_pred_test = model.predict(x_test)
y_pred_train = model.predict(x_train)

cat_mae_train = utils.MAE(y_train, y_pred_train)
cat_mae_test = utils.MAE(y_test, y_pred_test)

print(f"MAE LightGBM (train): {cat_mae_train:.5f}")
print(f"MAE LightGBM (test): {cat_mae_test:.5f}")

MAE LightGBM (train): 615476.77364
MAE LightGBM (test): 602012.34433
